In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
import seaborn as sns

pd.set_option('display.max_columns', None)



In [2]:
df = pd.read_csv(r'C:\Users\abhir\Projects\DS_Salary_Project\eda_data.csv')

# choose relevant columns 
df_model = df[['Rating', 'Size', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 
               'hourly', 'employer_provided', 'job_state', 'same_state', 'age', 'python_yn',
               'R_yn', 'spark', 'aws','excel', 'job_simple', 'seniority', 'desc_len', 
               'num_competitors', 'avg_salary']]

# get dummy data 
df_dum = pd.get_dummies(df_model)

In [3]:
X = df_dum.drop('avg_salary', axis =1)
y = df_dum.avg_salary.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Select only numeric columns for outlier detection, excluding dummy variables
numeric_columns = X_train.select_dtypes(include=['float64', 'int64'])

# If you have specific columns you know are dummy variables, you can exclude them explicitly
# For example, let's say 'dummy_col1' and 'dummy_col2' are your dummy variables
dummy_cols = ['Rating', 'Size', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 
               'hourly', 'employer_provided', 'job_state', 'same_state', 'age', 'python_yn',
               'R_yn', 'spark', 'aws','excel', 'job_simple', 'seniority', 'desc_len', 
               'num_competitors', 'avg_salary']
numeric_columns = numeric_columns.drop(columns=dummy_cols, errors='ignore')

# Calculate Q1 (25th percentile) and Q3 (75th percentile) for relevant numeric columns
Q1 = numeric_columns.quantile(0.25)
Q3 = numeric_columns.quantile(0.75)
IQR = Q3 - Q1

# Define outlier condition for numeric columns
outlier_condition = (numeric_columns < (Q1 - 1.5 * IQR)) | (numeric_columns > (Q3 + 1.5 * IQR))

# Identify outliers
outlier_indices = X_train[outlier_condition.any(axis=1)].index  # Get indices where any column is an outlier
print(f"Outlier Indices: {outlier_indices.tolist()}")



Outlier Indices: []


In [25]:
df_dum

,Rating,hourly,employer_provided,same_state,age,python_yn,R_yn,spark,aws,excel,desc_len,num_competitors,avg_salary,Size_-1,Size_1 to 50 employees,Size_10000+ employees,Size_1001 to 5000 employees,Size_201 to 500 employees,Size_5001 to 10000 employees,Size_501 to 1000 employees,Size_51 to 200 employees,Size_Unknown,Type of ownership_-1,Type of ownership_College / University,Type of ownership_Company - Private,Type of ownership_Company - Public,Type of ownership_Government,Type of ownership_Hospital,Type of ownership_Nonprofit Organization,Type of ownership_Other Organization,Type of ownership_School / School District,Type of ownership_Subsidiary or Business Segment,Type of ownership_Unknown,Industry_-1,Industry_Accounting,Industry_Advertising & Marketing,Industry_Aerospace & Defense,Industry_Architectural & Engineering Services,Industry_Auctions & Galleries,Industry_Banks & Credit Unions,Industry_Beauty & Personal Accessories Stores,Industry_Biotech & Pharmaceuticals,Industry_Brokerage Services,Industry_Colleges & Universities,Industry_Computer Hardware & Software,Industry_Construction,Industry_Consulting,Industry_Consumer Product Rental,Industry_Consumer Products Manufacturing,"Industry_Department, Clothing, & Shoe Stores",Industry_Education Training Services,Industry_Energy,Industry_Enterprise Software & Network Solutions,Industry_Farm Support Services,Industry_Federal Agencies,Industry_Financial Analytics & Research,Industry_Financial Transaction Processing,Industry_Food & Beverage Manufacturing,Industry_Gambling,Industry_Gas Stations,Industry_Health Care Products Manufacturing,Industry_Health Care Services & Hospitals,"Industry_Health, Beauty, & Fitness",Industry_IT Services,Industry_Industrial Manufacturing,Industry_Insurance Agencies & Brokerages,Industry_Insurance Carriers,Industry_Internet,Industry_Investment Banking & Asset Management,Industry_K-12 Education,Industry_Lending,Industry_Logistics & Supply Chain,Industry_Metals Brokers,Industry_Mining,Industry_Motion Picture Production & Distribution,Industry_Other Retail Stores,Industry_Real Estate,Industry_Religious Organizations,Industry_Research & Development,Industry_Security Services,Industry_Social Assistance,Industry_Sporting Goods Stores,Industry_Staffing & Outsourcing,Industry_Stock Exchanges,Industry_TV Broadcast & Cable Networks,Industry_Telecommunications Manufacturing,Industry_Telecommunications Services,Industry_Transportation Equipment Manufacturing,Industry_Transportation Management,Industry_Travel Agencies,Industry_Trucking,Industry_Video Games,Industry_Wholesale,Sector_-1,Sector_Accounting & Legal,Sector_Aerospace & Defense,Sector_Agriculture & Forestry,"Sector_Arts, Entertainment & Recreation",Sector_Biotech & Pharmaceuticals,Sector_Business Services,"Sector_Construction, Repair & Maintenance",Sector_Consumer Services,Sector_Education,Sector_Finance,Sector_Government,Sector_Health Care,Sector_Information Technology,Sector_Insurance,Sector_Manufacturing,Sector_Media,Sector_Mining & Metals,Sector_Non-Profit,"Sector_Oil, Gas, Energy & Utilities",Sector_Real Estate,Sector_Retail,Sector_Telecommunications,Sector_Transportation & Logistics,Sector_Travel & Tourism,Revenue_$1 to $2 billion (USD),Revenue_$1 to $5 million (USD),Revenue_$10 to $25 million (USD),Revenue_$10+ billion (USD),Revenue_$100 to $500 million (USD),Revenue_$2 to $5 billion (USD),Revenue_$25 to $50 million (USD),Revenue_$5 to $10 billion (USD),Revenue_$5 to $10 million (USD),Revenue_$50 to $100 million (USD),Revenue_$500 million to $1 billion (USD),Revenue_-1,Revenue_Less than $1 million (USD),Revenue_Unknown / Non-Applicable,job_state_AL,job_state_AZ,job_state_CA,job_state_CO,job_state_CT,job_state_DC,job_state_DE,job_state_FL,job_state_GA,job_state_IA,job_state_ID,job_state_IL,job_state_IN,job_state_KS,job_state_KY,job_state_LA,job_state_MA,job_state_MD,job_state_MI,job_state_MN,job_state_MO,job_state_NC,job_state_NE,job_state_NJ,job_state_NM,job_state_NY,job_state_OH,job_state_OR,job_stat

In [27]:
df[['Rating', 'Size', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 
               'hourly', 'employer_provided', 'job_state', 'same_state', 'age', 'python_yn',
               'R_yn', 'spark', 'aws','excel', 'job_simple', 'seniority', 'desc_len', 
               'num_competitors', 'avg_salary']]

,Rating,Size,Type of ownership,Industry,Sector,Revenue,hourly,employer_provided,job_state,same_state,age,python_yn,R_yn,spark,aws,excel,job_simple,seniority,desc_len,num_competitors,avg_salary
0,3.8,501 to 1000 employees,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),0,0,NM,0,47,1,0,0,0,1,data scientist,na,2536,1,72.0
1,3.4,10000+ employees,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),0,0,MD,0,36,1,0,0,0,0,data scientist,na,4783,1,87.5
2,4.8,501 to 1000 employees,Company - Private,Security Services,Business Services,$100 to $500 million (USD),0,0,FL,1,10,1,0,1,0,1,data scientist,na,3461,1,85.0
3,3.8,1001 to 5000 employees,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),0,0,WA,1,55,1,0,0,0,0,data scientist,na,3883,3,76.5
4,2.9,51 to 200 employees,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,0,0,NY,1,22,1,0,0,0,1,data scientist,na,2728,3,114.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,3.9,10000+ employees,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD),0,0,MA,0,190,0,0,0,1,0,na,senior,6162,3,84.5
738,4.4,1001 to 5000 employees,Company - Public,Internet,Information Technology,$100 to $500 million (USD),0,0,TN,0,14,1,0,1,1,0,data engineer,senior,6130,3,102.5
739,2.6,501 to 1000 employees,College / University,Colleges & Universities,Education,Unknown / Non-Applicable,0,0,PA,1,36,0,0,0,0,1,na,na,3078,1,73.5
740,3.2,1 to 50 employees,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),0,0,PA,0,-1,0,0,0,0,1,manager,na,1642,1,127.5
